**Content-Based:12 most popular items**



```
For users who have less than 2 purchased items, 12 most popular items are recommended
```



In [1]:
import time
import numpy as np
import pandas as pd

import multiprocessing as mp
from multiprocessing import Pool
from functools import partial

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
import os

In [4]:
root= '/gdrive/MyDrive/hnm/data/'
csv_train = os.path.join(root,'raw/transactions_train.csv')
csv_sub = os.path.join(root, 'raw/sample_submission.csv')
csv_users = os.path.join(root, 'raw/customers.csv')
csv_items = os.path.join(root, 'raw/articles.csv')

In [5]:
df = pd.read_csv(csv_train, dtype={'article_id': str})
df_sub = pd.read_csv(csv_sub)
df_sub.head

<bound method NDFrame.head of                                                customer_id  \
0        00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...   
1        0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...   
2        000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...   
3        00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...   
4        00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...   
...                                                    ...   
1371975  ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...   
1371976  ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...   
1371977  ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...   
1371978  ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...   
1371979  ffffd9ac14e89946416d80e791d064701994755c3ab686...   

                                                prediction  
0        0706016001 0706016002 0372860001 0610776002 07...  
1        0706016001 0706016002 0372860001 0610776002 07...  
2        0706016001 0706016002 0372860001 

In [6]:
df.t_dat.max()

'2020-09-22'

In [7]:
top_12_items = df[df['t_dat'] > '2020-09-01'].groupby('article_id')['customer_id'].nunique().sort_values(ascending=False).head(12).index.tolist()
top_12_items

['0918522001',
 '0909370001',
 '0751471001',
 '0924243001',
 '0918292001',
 '0915529003',
 '0915526001',
 '0751471043',
 '0448509014',
 '0706016001',
 '0863595006',
 '0865799006']

In [11]:
df_sub['prediction'] =  ' '.join(top_12_items)
df_sub.columns=['customer_id','prediction']
df_sub.iloc[1]


customer_id    0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...
prediction     0918522001 0909370001 0751471001 0924243001 09...
Name: 1, dtype: object

In [20]:
submission=pd.read_csv('/submission-3.csv')

submission.columns=['customer_id','prediction']
submission.head
#submission=submission.dropna()

<bound method NDFrame.head of                                              customer_id  \
0      0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...   
1      000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...   
2      00007d2de826758b65a93dd24ce629ed66842531df6699...   
3      0003abe64294e66a6310c3436fa9e5b754cc5603deef4f...   
4      0004068f54dbe1c7054b23c615edc5f733a508ecc54930...   
...                                                  ...   
18720  fff0ac18093a702a0a06f4cc76582632df3ede9a36556e...   
18721  fff15526121f7d914a54784e68761a1d30b7547e355573...   
18722  fff3e75605ec575be9b95eda1e6557299e81bba12668d7...   
18723  fff4b145d7469e023b147b0f8375c565b1be4394498779...   
18724  fff627c97a69e53afb4a2b49a3ebf7fa06660afaac959b...   

                                              prediction  
0      [578430002, 639552001, 583558001, 617620003, 5...  
1      [663713001, 541518023, 464277014, 611415005, 5...  
2      [505221004, 685687003, 685687004, 685687001, 5...  
3      [53131

In [28]:
a=submission['customer_id']
df_sub=df_sub.drop(df_sub[df_sub['customer_id'].isin(a)].index)

In [29]:
df_sub.shape

(1353255, 2)

In [30]:
df_union=pd.concat([df_sub,submission])

In [42]:
df_union.to_csv('final_submission')

In [41]:
df_union=df_union.set_index('customer_id')
df_union.head

<bound method NDFrame.head of                                                                                            prediction
customer_id                                                                                          
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d...  0918522001 0909370001 0751471001 0924243001 09...
00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c...  0918522001 0909370001 0751471001 0924243001 09...
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe...  0918522001 0909370001 0751471001 0924243001 09...
000064249685c11552da43ef22a5030f35a147f723d5b02...  0918522001 0909370001 0751471001 0924243001 09...
0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d8...  0918522001 0909370001 0751471001 0924243001 09...
...                                                                                               ...
fff0ac18093a702a0a06f4cc76582632df3ede9a36556e3...  [692454002, 617322003, 620405001, 613456009, 6...
fff15526121f7d914a54784e68761a1d30b7547e3555738...  

In [35]:
df_union.head

<bound method NDFrame.head of                                              customer_id  \
0      00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...   
3      00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...   
4      00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...   
5      000064249685c11552da43ef22a5030f35a147f723d5b0...   
6      0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...   
...                                                  ...   
18720  fff0ac18093a702a0a06f4cc76582632df3ede9a36556e...   
18721  fff15526121f7d914a54784e68761a1d30b7547e355573...   
18722  fff3e75605ec575be9b95eda1e6557299e81bba12668d7...   
18723  fff4b145d7469e023b147b0f8375c565b1be4394498779...   
18724  fff627c97a69e53afb4a2b49a3ebf7fa06660afaac959b...   

                                              prediction  
0      0918522001 0909370001 0751471001 0924243001 09...  
3      0918522001 0909370001 0751471001 0924243001 09...  
4      0918522001 0909370001 0751471001 0924243001 09...  
5      091852